In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework') \
    .getOrCreate()

22/02/28 01:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Question 1

In [4]:
spark.version

'3.0.3'

### Question 2

In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-24 01:32:10--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.139.1
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.139.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv.1’

.1                   10%[=>                  ]  72.05M  10.6MB/s    eta 78s    ^C


In [ ]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [5]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [6]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet("fhvhv/2021/02")

In [ ]:
!du -h "fhvhv/2021/02"

210M	fhvhv/2021/02


### Question 3

In [7]:
df = spark.read.parquet("fhvhv/2021/02")

In [8]:
df.registerTempTable("fhvhv")

In [9]:
df.take(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02887', pickup_datetime=datetime.datetime(2021, 2, 6, 1, 18, 35), dropoff_datetime=datetime.datetime(2021, 2, 6, 1, 40, 34), PULocationID=163, DOLocationID=235, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 5, 7, 13, 6), dropoff_datetime=datetime.datetime(2021, 2, 5, 7, 31, 56), PULocationID=225, DOLocationID=181, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02869', pickup_datetime=datetime.datetime(2021, 2, 4, 16, 56, 52), dropoff_datetime=datetime.datetime(2021, 2, 4, 17, 21, 36), PULocationID=260, DOLocationID=95, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02871', pickup_datetime=datetime.datetime(2021, 2, 3, 18, 34, 17), dropoff_datetime=datetime.datetime(2021, 2, 3, 18, 57, 12), PULocationID=235, DOLocationID=60, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02869', pickup_

In [10]:
spark.sql("""
select count(1) as feb_15_trips from fhvhv
where date_format(pickup_datetime, 'yyyyMMdd') = '20210215'
""").show()

+------------+
|feb_15_trips|
+------------+
|      367170|
+------------+



### Question 4

In [11]:
spark.sql("""
select pickup_datetime, dropoff_datetime, unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) as trip_duration
from fhvhv
order by trip_duration desc
limit 1
""").show()

+-------------------+-------------------+-------------+
|    pickup_datetime|   dropoff_datetime|trip_duration|
+-------------------+-------------------+-------------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|        75540|
+-------------------+-------------------+-------------+



### Question 5

In [12]:
spark.sql("""
select dispatching_base_num, count(1) as count_base_num
from fhvhv
group by dispatching_base_num
order by count_base_num desc
limit 1
""").show()

+--------------------+--------------+
|dispatching_base_num|count_base_num|
+--------------------+--------------+
|              B02510|       3233664|
+--------------------+--------------+



Number of Stages: 3

### Question 6

In [38]:
# Downloading Zones lookup file
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-24 02:33:26--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.85.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.85.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-24 02:33:27 (46.0 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [13]:
zones_schema = types.StructType([
    types.StructField("LocationID", types.IntegerType(), nullable=True),
    types.StructField("Borough", types.StringType(), nullable=True),
    types.StructField("Zone", types.StringType(), nullable=True),
    types.StructField("service_zone", types.StringType(), nullable=True)
])

In [14]:
zones_df = spark.read.csv("taxi+_zone_lookup.csv", schema = zones_schema, header=True)

In [15]:
zones_df.take(5)

[Row(LocationID=1, Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID=2, Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID=3, Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID=4, Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID=5, Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [16]:
zones_df.registerTempTable("zones")

In [17]:
spark.sql("select * from zones limit 5").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [18]:
pickup_dropoff_df = spark.sql("""
select coalesce(pickup_zone.Zone, 'Unknown') || '/' || coalesce(dropoff_zone.Zone, 'Unknown') as pickup_dropoff_pair,
count(1) as count_pickup_dropoff_pair
from fhvhv
left join zones as pickup_zone on fhvhv.PULocationID = pickup_zone.LocationID
left join zones as dropoff_zone on fhvhv.DOLocationID = dropoff_zone.LocationID
group by pickup_dropoff_pair
order by count_pickup_dropoff_pair desc
limit 5
""")

In [19]:
pickup_dropoff_df.collect()

[Row(pickup_dropoff_pair='East New York/East New York', count_pickup_dropoff_pair=45041),
 Row(pickup_dropoff_pair='Borough Park/Borough Park', count_pickup_dropoff_pair=37329),
 Row(pickup_dropoff_pair='Canarsie/Canarsie', count_pickup_dropoff_pair=28026),
 Row(pickup_dropoff_pair='Crown Heights North/Crown Heights North', count_pickup_dropoff_pair=25976),
 Row(pickup_dropoff_pair='Bay Ridge/Bay Ridge', count_pickup_dropoff_pair=17934)]

### Join Type

Broadcast Hash Join
Stages: 2